In [4]:
!python ../../scripts/iuxray/extract_findings_and_impressions.py

Loading file /mnt/workspace/iu-x-ray/dataset/reports/reports.min.json ...
Extracing and merging findings and impressions ...
Saving to /home/pamessina/medvqa-workspace/cache/iuxray/findings+impression.txt ...
done!


In [5]:
!python ../../scripts/mimiccxr/extract_findings_and_impressions.py

Loading MIMIC-CXR's report file paths ...
227835it [03:10, 1197.91it/s] 
Saving findings + impressions to /home/pamessina/medvqa-workspace/cache/mimiccxr/findings+impression.txt ...
100%|████████████████████████████████| 227835/227835 [00:10<00:00, 21755.81it/s]
Num failed paths = 649
Done!


In [6]:
from importlib import reload

In [7]:
from medvqa.utils.files import read_lines_from_txt
from medvqa.utils.common import CACHE_DIR
from medvqa.datasets.qa_pairs_extractor import QuestionAnswerExtractor
import medvqa
import re
import os
import random
import math
from tqdm import tqdm

In [8]:
reload(medvqa.datasets.qa_pairs_extractor)

<module 'medvqa.datasets.qa_pairs_extractor' from '/home/pamessina/medvqa/medvqa/datasets/qa_pairs_extractor.py'>

In [9]:
iuxray_raw_reports = read_lines_from_txt(os.path.join(CACHE_DIR, 'iuxray', 'findings+impression.txt'))

In [10]:
mimiccxr_raw_reports = read_lines_from_txt(os.path.join(CACHE_DIR, 'mimiccxr', 'findings+impression.txt'))

In [11]:
len(iuxray_raw_reports), len(mimiccxr_raw_reports)

(3927, 227186)

In [12]:
def print_qa_pairs(qa_pairs):
    for k, v in qa_pairs.items():
        print('------------')
        print('Q:', k)
        for s in v:
            print('A:',s)

In [13]:
qa_extractor = QuestionAnswerExtractor()

In [14]:
connectors = ['when', 'and', 'but', 'however', 'although', 'should', 'since', 'unless', 'nevertheless']
exp = '('+'|'.join(f'\\b{x}\\b' for x in connectors)+'|[,;.:])'
full_exp = f'{exp}+(\\s+{exp}+)*'
sep_regex = re.compile(full_exp, re.IGNORECASE)
sep_regex

re.compile(r'(\bwhen\b|\band\b|\bbut\b|\bhowever\b|\balthough\b|\bshould\b|\bsince\b|\bunless\b|\bnevertheless\b|[,;.:])+(\s+(\bwhen\b|\band\b|\bbut\b|\bhowever\b|\balthough\b|\bshould\b|\bsince\b|\bunless\b|\bnevertheless\b|[,;.:])+)*',
re.IGNORECASE|re.UNICODE)

In [15]:
def clean_sentence(self, sentence):
    last_sep = None
    i = 0
    n = len(sentence)
    sentence = self._apply_replacements(sentence)
    clean_sentence = ''
    prepend_to_next_chunk = None
    for match in sep_regex.finditer(sentence):
        span = match.span()
        if i < span[0]:
            chunk = sentence[i:span[0]].strip()
            if prepend_to_next_chunk:
                if len(clean_sentence) > 0:
                    chunk = prepend_to_next_chunk + chunk
                prepend_to_next_chunk = None
            if self._is_relevant(chunk):
                if len(clean_sentence) > 0:
                    if last_sep:
                        clean_sentence += ' ' + last_sep
                    clean_sentence += ' '                    
                clean_sentence += chunk
            print('chunk =', chunk)
        i = span[1]
        last_sep = sentence[span[0]:span[1]]
        print('last_sep =', last_sep)
        if last_sep not in ('and', 'but', ':'):
            if last_sep == ', however,':
                last_sep = None
            else:
                done = False
                for x in ('should', 'since', 'unless', 'when'):
                    if x in last_sep:
                        prepend_to_next_chunk = last_sep + ' '
                        last_sep = None
                        done = True
                        break
                if not done:
                    last_sep = ','
    if i < n:
        chunk = sentence[i:].strip()
        if prepend_to_next_chunk:
            chunk = prepend_to_next_chunk + chunk
        if self._is_relevant(chunk):
            if len(clean_sentence) > 0:
                if last_sep:
                    clean_sentence += ' ' + last_sep
                clean_sentence += ' '
            clean_sentence += chunk
        print('chunk =', chunk)
    return clean_sentence

In [16]:
# _s = 'AP chest compared to through : Pulmonary vascular congestion and mediastinal venous engorgement have progressed since .'
_s = 'pa and lateral views of the chest provided demonstrate clear lungs without focal consolidation , effusion or pneumothorax . the cardiomediastinal silhouette is normal . bony structures are intact . no free air below the right hemidiaphragm .'
for x in sep_regex.finditer(_s):
    print(str(x))
print("$$$$"+clean_sentence(qa_extractor, _s)+"$$$$")
print("$$$$"+qa_extractor.clean_sentence(_s)+"$$$$")

<_sre.SRE_Match object; span=(3, 6), match='and'>
<_sre.SRE_Match object; span=(95, 96), match=','>
<_sre.SRE_Match object; span=(122, 123), match='.'>
<_sre.SRE_Match object; span=(167, 168), match='.'>
<_sre.SRE_Match object; span=(196, 197), match='.'>
<_sre.SRE_Match object; span=(240, 241), match='.'>
chunk = pa
last_sep = and
chunk = lateral views of the chest provided demonstrate clear lungs without focal consolidation
last_sep = ,
chunk = effusion or pneumothorax
last_sep = .
chunk = the cardiomediastinal silhouette is normal
last_sep = .
chunk = bony structures are intact
last_sep = .
chunk = no free air below the right hemidiaphragm
last_sep = .
$$$$lateral views of the chest provided demonstrate clear lungs without focal consolidation , effusion or pneumothorax , the cardiomediastinal silhouette is normal , bony structures are intact , no free air below the right hemidiaphragm$$$$
$$$$lateral views of the chest provided demonstrate clear lungs without focal consolidation , e

In [17]:
from nltk.tokenize import sent_tokenize

In [18]:
sent_tokenize('Mild-to-moderate pulmonary edema is new. Right lower lobe opacity has improved consistent with improving atelectasis. Small bilateral pleural effusions are unchanged. Opacity in the left apex and irregularity of the left upper ribs corresponds to mass seen in prior CTA. Cardiomegaly is stable. Increasing opacity in the left lower lobe is consistent with increasing atelectasis and pleural effusion. Spinal hardware is again noted.')

['Mild-to-moderate pulmonary edema is new.',
 'Right lower lobe opacity has improved consistent with improving atelectasis.',
 'Small bilateral pleural effusions are unchanged.',
 'Opacity in the left apex and irregularity of the left upper ribs corresponds to mass seen in prior CTA.',
 'Cardiomegaly is stable.',
 'Increasing opacity in the left lower lobe is consistent with increasing atelectasis and pleural effusion.',
 'Spinal hardware is again noted.']

In [19]:
tmp = qa_extractor.generate_qa_pairs('Previous moderate left pleural effusion and left lower lobe collapse have improved has the patient had thoracentesis?  There is no pneumothorax. Moderate enlarged of the cardiac silhouette is clear due to cardiomegaly and/or pericardial effusion and there is new mild interstitial edema in the right lower lung. Hemodialysis catheter ends in the right atrium as before.', debug=True)
print_qa_pairs(tmp)

------------
Q: pleural space?
A: Previous moderate left pleural effusion
A: Moderate enlarged of the cardiac silhouette is clear due to cardiomegaly and /or pericardial effusion and there is new mild interstitial edema in the right lower lung
------------
Q: pneumothorax?
A: There is no pneumothorax
------------
Q: lungs?
A: Moderate enlarged of the cardiac silhouette is clear due to cardiomegaly and /or pericardial effusion and there is new mild interstitial edema in the right lower lung
------------
Q: heart?
A: Moderate enlarged of the cardiac silhouette is clear due to cardiomegaly and /or pericardial effusion and there is new mild interstitial edema in the right lower lung
A: Hemodialysis catheter ends in the right atrium as before
------------
Q: cardiomegaly?
A: Moderate enlarged of the cardiac silhouette is clear due to cardiomegaly and /or pericardial effusion and there is new mild interstitial edema in the right lower lung
------------
Q: cardiac silhouette?
A: Moderate enla

In [20]:
report = random.choice(mimiccxr_raw_reports)
print(report)
print('=============================')
tmp = qa_extractor.generate_qa_pairs(report, debug=True)
print_qa_pairs(tmp)

In comparison with the study of , the right subclavian PICC line has been pulled back, though the tip still remains in the left brachiocephalic vein. Other monitoring and support devices are unchanged. The bilateral pulmonary opacifications have decreased, especially on the right. Nevertheless, in the appropriate clinical setting, aspiration pneumonia at the left base would have to be seriously considered.
------------
Q: tubes and lines?
A: the right subclavian PICC line has been pulled back , though the tip still remains in the left brachiocephalic vein
------------
Q: support devices and foreign bodies?
A: the right subclavian PICC line has been pulled back , though the tip still remains in the left brachiocephalic vein
A: Other monitoring and support devices are unchanged
------------
Q: opacities?
A: The bilateral pulmonary opacifications have decreased , especially on the right
------------
Q: lungs?
A: in the appropriate clinical setting , aspiration pneumonia at the left base w

In [21]:
tmp = qa_extractor.generate_qa_pairs_compact_version(report)
tmp

{'sentences': ['the right subclavian PICC line has been pulled back , though the tip still remains in the left brachiocephalic vein',
  'Other monitoring and support devices are unchanged',
  'The bilateral pulmonary opacifications have decreased , especially on the right',
  'in the appropriate clinical setting , aspiration pneumonia at the left base would have to be seriously considered'],
 'invalid': [],
 'unmatched': [],
 'matched': [0, 1, 2, 3],
 'qa': {'95': [0], '87': [0, 1], '65': [2], '59': [3], '71': [3], '10': [3]},
 'question_ids': [95, 87, 65, 59, 71, 10]}

In [22]:
_s = 'The tip of the endotracheal tube projects approximately 3.5 cm above the carina and is within normal limits.'
print("$$$$"+qa_extractor.clean_sentence(_s)+"$$$$")

$$$$The tip of the endotracheal tube projects approximately NUMBER cm above the carina and is normal$$$$


In [23]:
len(qa_extractor.questions), qa_extractor.questions

(97,
 ['ARDS?',
  'COPD?',
  'abscess and cavitation?',
  'adenopathy?',
  'air collections?',
  'air space disease?',
  'air-fluid level?',
  'airways?',
  'apical zone?',
  'ascites?',
  'aspiration?',
  'atelectasis?',
  'azygos lobe?',
  'azygos vein?',
  'bleeding?',
  'blurring?',
  'bones?',
  'bowel obstruction and loops?',
  'bowel?',
  'bronchiectasis?',
  'bronchogram?',
  'bronchovascular crowding?',
  'bronchus?',
  'bullae and blebs?',
  'calcification?',
  'cancer or tumor?',
  'cardiac silhouette?',
  'cardiomegaly?',
  'carina?',
  'cholelithiasis?',
  'colon?',
  'congestive heart failure CHF?',
  'consolidation?',
  'contrast?',
  'densities?',
  'diaphragm?',
  'edema?',
  'emphysema?',
  'esophagus?',
  'fibrosis?',
  'fissures?',
  'fluid overload?',
  'fluid?',
  'fractures?',
  'free air?',
  'gas distension?',
  'granumola?',
  'haze?',
  'heart?',
  'hernias?',
  'infection?',
  'infiltrate?',
  'interstitial lung disease?',
  'kerley lines?',
  'lesions?',
  

In [24]:
tmp = qa_extractor.search_by_unmatched(0, mimiccxr_raw_reports, 100)
x = tmp[1]
tmp

(4, 33058)

In [25]:
qa_extractor.generate_qa_pairs_compact_version(mimiccxr_raw_reports[x])

{'sentences': ['there is no relevant change',
  'The right lower hemithorax has minimally increased in transparency , the left hemithorax has slightly decreased in transparency',
  'These changes are likely caused by changes in patient position',
  'Unchanged overall severity of pulmonary pathology',
  'Constant size of the cardiac silhouette',
  'Unchanged appearance of the cardiac monitoring and support devices'],
 'invalid': [],
 'unmatched': [0, 1, 2, 3],
 'matched': [4, 5],
 'qa': {'26': [4], '87': [5]},
 'question_ids': [26, 87]}

In [26]:
# _report = ""
# print(_report)
# _qa_pairs = qa_generator.generate_question_answer_pairs(_report)
print(mimiccxr_raw_reports[x])
print("======================")
_qa_pairs = qa_extractor.generate_qa_pairs(mimiccxr_raw_reports[x], debug=True)
# _qa_pairs = qa_generator.remove_almost_duplicate_sentences(_qa_pairs)
print_qa_pairs(_qa_pairs)

As compared to the previous radiograph, there is no relevant change. The right lower hemithorax has minimally increased in transparency, the left hemithorax has slightly decreased in transparency. These changes are likely caused by changes in patient position. Unchanged overall severity of pulmonary pathology. Constant size of the cardiac silhouette. Unchanged appearance of the cardiac monitoring and support devices.
** not captured: there is no relevant change
** not captured: The right lower hemithorax has minimally increased in transparency , the left hemithorax has slightly decreased in transparency
** not captured: These changes are likely caused by changes in patient position
** not captured: Unchanged overall severity of pulmonary pathology
------------
Q: cardiac silhouette?
A: Constant size of the cardiac silhouette
------------
Q: support devices and foreign bodies?
A: Unchanged appearance of the cardiac monitoring and support devices


In [27]:
def get_informative_unmatched_sentences(corpus, qa_extractor, n_samples = 2000):
    
    indices = range(len(corpus))
    if len(corpus) > n_samples:
        indices = random.sample(indices, n_samples)
        
    unmatched_sentences = set()
    word2freq = dict()
    
    for i in tqdm(indices):
        for s in qa_extractor.get_unmatched_sentences(corpus[i]):
            unmatched_sentences.add(s)
            for w in re.split(r",?\s+", s):
                word2freq[w] = word2freq.get(w, 0) + 1
    
    max_freq = max(word2freq.values())    
    unmatched_sentences = list(unmatched_sentences)
    
    def score(s):
        score_max = 0
        score_sum = 0
        cnt = 0
        for w in re.split(r",?\s+", s):
            if w not in qa_extractor.stopwords:
                tmp = max_freq - word2freq.get(w, 0)
                score_max = max(tmp, score_max)
                score_sum += tmp
                cnt += 1
        return score_max * 0.4 + (score_sum / math.sqrt(max(cnt,1))) * 0.6
    
    unmatched_sentences.sort(key=score, reverse=True)
    return unmatched_sentences

In [28]:
unmatched_sentences = get_informative_unmatched_sentences(mimiccxr_raw_reports, qa_extractor, n_samples=1000)

100%|██████████| 1000/1000 [00:05<00:00, 182.62it/s]


In [29]:
len(unmatched_sentences)

198

In [30]:
qa_extractor.reload()

In [31]:
print_qa_pairs(qa_extractor.generate_qa_pairs('. '.join(unmatched_sentences[0:40]), debug=True))

** not captured: The multifocal patchy abnormalities identified in chest CT are not particularly well seen on current chest radiograph
** not captured: Small small to moderate residual left hemothorax has continued to decrease
** not captured: An additional right lateral decubitus view would be helpful for clarification
** not captured: the patient has an outside CT scan , which has not yet been uploaded for reference
** not captured: One of the images shows the midline and left chest , excludes the right
** not captured: Oblique views might be able to demonstrate both of these areas to better advantage
** not captured: MRI of the chest may be useful in differentiating thymic hyperplasia from lymphoma
** not captured: long term stability suggest a benign etiology
** not captured: The remainder of the examination is unchanged compared to the prior radiograph
** not captured: Findings may represent underpenetration secondary to technique
** not captured: Disease at the right cardiophreni

In [32]:
!python ../../scripts/iuxray/generate_qa_pairs_from_reports.py

Loading original reports from /mnt/workspace/iu-x-ray/dataset/reports/reports.min.json
Processing each report ...
100%|██████████████████████████████████████| 3955/3955 [00:31<00:00, 123.88it/s]
Saving qa adapted reports to /home/pamessina/medvqa-workspace/cache/iuxray/qa_adapted_reports__20220629_042239.json
Done!


In [33]:
!python ../../scripts/mimiccxr/generate_qa_pairs_from_reports.py

Processing MIMIC-CXR's original reports ...
198369it [38:14, 85.93it/s] 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



227835it [44:01, 86.25it/s] 
Saving qa adapted reports to /home/pamessina/medvqa-workspace/cache/mimiccxr/qa_adapted_reports__20220629_050643.json
Done!


## Generate summary file

In [34]:
from medvqa.utils.files import load_json_file

In [35]:
iuxray_qa_adapted_reports = load_json_file('/home/pamessina/medvqa-workspace/cache/iuxray/qa_adapted_reports__20220629_042239.json')

In [36]:
mimiccxr_qa_adapted_reports = load_json_file('/home/pamessina/medvqa-workspace/cache/mimiccxr/qa_adapted_reports__20220629_050643.json')

In [37]:
len(iuxray_qa_adapted_reports['questions']), len(iuxray_qa_adapted_reports['reports'])

(97, 3927)

In [38]:
len(mimiccxr_qa_adapted_reports['questions']), len(mimiccxr_qa_adapted_reports['reports'])

(97, 227835)

In [39]:
iuxray_qa_adapted_reports['reports'][0]

{'sentences': ['The lungs are hyperexpanded , with increased AP diameter of the chest',
  'The cardiomediastinal silhouette is stable and normal',
  'There is no pneumothorax or large pleural effusion',
  'No acute cardiopulmonary abnormality',
  'Chronic changes consistent with emphysema'],
 'invalid': [],
 'unmatched': [],
 'matched': [0, 1, 2, 3, 4],
 'qa': {'59': [0, 3],
  '26': [1],
  '63': [1],
  '68': [2],
  '73': [2],
  '48': [3],
  '37': [4]},
 'question_ids': [59, 26, 63, 68, 73, 48, 37],
 'filename': '2509.xml'}

In [40]:
mimiccxr_qa_adapted_reports['reports'][0]

{'sentences': ['Heart size is normal',
  'There are mild aortic knob calcifications',
  'Cardiomediastinal silhouette and hilar contours are otherwise unremarkable',
  'Lungs are clear',
  'Pleural surfaces are clear without effusion or pneumothorax',
  'No acute cardiopulmonary abnormality'],
 'invalid': [],
 'unmatched': [],
 'matched': [0, 1, 2, 3, 4, 5],
 'qa': {'48': [0, 5],
  '90': [1],
  '24': [1],
  '75': [2],
  '26': [2],
  '63': [2],
  '59': [3, 5],
  '68': [4],
  '73': [4]},
 'question_ids': [48, 90, 24, 75, 26, 63, 59, 68, 73],
 'filepath': '/mnt/data/mimic-cxr/files/p10/p10703179/s58829627.txt'}

In [41]:
def generate_summary_file(dataset, k, kk, output_path):
    
    q2a = [[] for _ in range(len(dataset['questions']))]
    n_invalid = 0
    unmatched_sentences = []
    for ri, report in enumerate(dataset['reports']):
        s = report['sentences']
        for q_idx, a_idxs in report['qa'].items():
            q_idx = int(q_idx)
            a_len = sum(len(s[i]) for i in a_idxs)
            q2a[q_idx].append((a_len, ri))
        n_invalid += len(report['invalid'])
        unmatched_sentences.extend(s[i] for i in report['unmatched'])
            
    sorted_questions = sorted([(len(a), i) for i,a in enumerate(q2a)], reverse=True)    
    
    with open(output_path, 'w') as file:
        file.write('Preguntas ordenadas por frecuencia (decreciente)\n\n')
        for f, q in sorted_questions:
            file.write('%d, %s\n' % (f, dataset['questions'][q]))
        file.write('\n')
        file.write('Ejemplos de respuestas por pregunta:')
        for i,(_,q) in enumerate(sorted_questions):
            answers = q2a[q]
            answers.sort()            
            tmp = []
            for _, a in answers:
                report = dataset['reports'][a]
                sentences = report['sentences']
                a_idxs = report['qa'][str(q)]
                answer = '. '.join(sentences[i] for i in a_idxs)
                tmp.append(answer)
            answers = tmp            
            n = len(answers)
            print("n=%d, %s" % (n, dataset['questions'][q]))
            if n > k:
                sample = []
                for j in range(k):
                    sample.append(answers[random.randint(int(n*j/k), int(n*(j+1)/k)-1)])
            else:
                sample = answers
            file.write('\n\n%d) %s:\n\n' % (i+1, dataset['questions'][q]))
            for a in sample:
                file.write('    A: %s\n' % a)
        file.write('\n\nTotal oraciones inválidas: %d\n' % n_invalid)
        file.write('Total oraciones sin match: %d\n' % len(unmatched_sentences))
        file.write('\n\nEjemplos de oraciones válidas que no hicieron match con ninguna pregunta\n\n')
        unmatched_sentences = list(set(unmatched_sentences))
        unmatched_sentences.sort(key=lambda s:len(s))
        nn = len(unmatched_sentences)
        for j in range(kk):
            file.write('    %s\n' % unmatched_sentences[random.randint(int(nn*j/kk), int(nn*(j+1)/kk)-1)])

In [42]:
generate_summary_file(iuxray_qa_adapted_reports, 10, 120, '../iuxray/iuxray_qa_dataset_summary_file.txt')

n=3549, lungs?
n=3369, heart?
n=3106, pleural space?
n=2689, pneumothorax?
n=2358, mediastinum?
n=1960, bones?
n=1242, consolidation?
n=1184, cardiac silhouette?
n=815, pulmonary vascularity?
n=756, opacities?
n=601, calcification?
n=552, air space disease?
n=457, granumola?
n=439, support devices and foreign bodies?
n=431, infiltrate?
n=397, thoracic aorta?
n=395, atelectasis?
n=354, edema?
n=338, nodules?
n=305, cardiomegaly?
n=272, surgery?
n=266, pneumonia?
n=249, pulmonary hila?
n=244, diaphragm?
n=234, scarring?
n=226, tortuosity?
n=220, interstitial lung disease?
n=219, tubes and lines?
n=204, fractures?
n=193, ribs?
n=187, masses?
n=155, prominence?
n=149, densities?
n=145, soft tissue?
n=122, emphysema?
n=112, apical zone?
n=111, trachea?
n=103, bronchus?
n=100, adenopathy?
n=89, markings?
n=83, bronchovascular crowding?
n=74, fluid?
n=70, air collections?
n=68, infection?
n=64, free air?
n=62, lesions?
n=56, hernias?
n=55, vena cava?
n=49, COPD?
n=48, lung parenchyma?
n=44, l

In [43]:
generate_summary_file(mimiccxr_qa_adapted_reports, 10, 120, '../mimiccxr/mimiccxr_qa_dataset_summary_file.txt')

n=192912, lungs?
n=184641, pleural space?
n=144632, pneumothorax?
n=129627, heart?
n=118715, mediastinum?
n=108012, support devices and foreign bodies?
n=99016, cardiac silhouette?
n=86835, tubes and lines?
n=73530, bones?
n=71347, opacities?
n=70728, consolidation?
n=69741, atelectasis?
n=65994, edema?
n=54761, pulmonary hila?
n=54217, pneumonia?
n=54173, pulmonary vascularity?
n=32181, cardiomegaly?
n=27947, surgery?
n=25929, diaphragm?
n=22360, thoracic aorta?
n=20568, vena cava?
n=17330, stomach?
n=16369, trachea?
n=15835, interstitial lung disease?
n=14581, calcification?
n=14342, apical zone?
n=13903, fractures?
n=13804, infection?
n=12029, carina?
n=11713, lung parenchyma?
n=11553, ribs?
n=11388, prominence?
n=11185, tortuosity?
n=9608, free air?
n=9366, fluid?
n=8896, bronchus?
n=8551, aspiration?
n=7848, nodules?
n=7839, scarring?
n=7339, densities?
n=5642, emphysema?
n=5552, markings?
n=5366, congestive heart failure CHF?
n=4861, infiltrate?
n=4632, esophagus?
n=4403, masses?